In [23]:
import pandas as pd
import geopandas as gpd
import transbigdata as tbd  # 假设用来处理GPS与网格转换的模块
import ast
import numpy as np
def capacity(station_info_path):
    def process_station_data(station_info_path, step_length=300):
            # Load station info
            station_info = pd.read_csv(station_info_path)
            station_info = station_info[station_info['num_current_car'] > 0]

            # Extract relevant columns and clean data
            station_info_table = station_info[
                ['station_id', 'lon', 'lat', 'max_capacity', 'charge_speed_station']].drop_duplicates().copy()
            station_info = station_info[['station_id', 'time', 'current_car', 'waiting_car']]

            # Convert time and handle current/waiting car columns
            station_info['time'] = pd.to_datetime(station_info['time'])
            station_info['current_car'] = station_info['current_car'].apply(lambda a: ast.literal_eval(a))
            station_info['waiting_car'] = station_info['waiting_car'].apply(lambda a: ast.literal_eval(a))
            station_info.sort_values(by=['station_id', 'time'], inplace=True)

            # Process current car infos
            current_car_infos = station_info[['station_id', 'time', 'current_car']].explode('current_car')
            current_car_infos = current_car_infos[~current_car_infos['current_car'].isnull()]
            current_car_infos = current_car_infos.sort_values(by=['current_car', 'time'])[
                ['current_car', 'time', 'station_id']]

            # Process waiting car infos
            waiting_car_infos = station_info[['station_id', 'time', 'waiting_car']].explode('waiting_car')
            waiting_car_infos = waiting_car_infos[~waiting_car_infos['waiting_car'].isnull()]
            waiting_car_infos = waiting_car_infos.sort_values(by=['waiting_car', 'time'])[
                ['waiting_car', 'time', 'station_id']]

            # Helper function to generate charging or waiting orders
            def get_charging_order(car_infos, step_length):
                car_infos['timegap'] = car_infos['time'].diff().dt.total_seconds().fillna(1000000).astype(int)
                car_infos['order_id'] = (car_infos['timegap'] > step_length).cumsum()
                charge_info_s = car_infos.groupby(['current_car', 'order_id']).first().reset_index()
                charge_info_e = car_infos.groupby(['current_car', 'order_id']).last().reset_index()
                charging_order = pd.merge(charge_info_s, charge_info_e, on=['current_car', 'order_id', 'station_id'])
                charging_order = charging_order[['current_car', 'order_id', 'time_x', 'time_y', 'station_id']]
                charging_order.columns = ['carid', 'order_id', 'stime', 'etime', 'station_id']
                charging_order['duration'] = (charging_order['etime'] - charging_order['stime']).dt.total_seconds()
                charging_order = charging_order[charging_order['duration'] > 0]
                return charging_order

            # Generate charging and waiting orders
            charging_orders = get_charging_order(current_car_infos, step_length)
            waiting_orders = get_charging_order(waiting_car_infos.rename(columns={'waiting_car': 'current_car'}),
                                                step_length)

            # Merge station info details into orders
            charging_orders = pd.merge(charging_orders, station_info_table)
            waiting_orders = pd.merge(waiting_orders, station_info_table)

            # Save charging and waiting orders to CSV
            # charging_orders.to_csv('output/charging_orders_from_station.csv', index=False)
            # waiting_orders.to_csv('output/waiting_orders_from_station.csv', index=False)

            return charging_orders, waiting_orders

    def split_day(df):
        df["stime"]=pd.to_datetime(df["stime"])
        df["etime"]=pd.to_datetime(df["etime"])
        same_day_mask = (df['stime'].dt.date == df['etime'].dt.date)

        # 创建一个新的数据框，存放拆分后的数据
        split_data = []

        # 处理跨天的数据
        for index, row in df[~same_day_mask].iterrows():
            # 创建两行数据，一行的 'time_end' 是当天的 23:59:59，另一行的 'time_start' 是次日的 00:00:00i
            row1 = row.copy()
            row2 = row.copy()
            
            row1['etime'] = row1['stime'].replace(hour=23, minute=59, second=59)
            row2['stime'] = row1['etime'] + pd.Timedelta(seconds=1)
            
            split_data.extend([row1, row2])

        # 将拆分后的数据添加回原数据框，并且删除原数据框中跨天的数据
        df = pd.concat([df[same_day_mask], pd.DataFrame(split_data)], ignore_index=True)

        df1=df
        df1['duration'] = (df1['etime'] - df1['stime']).dt.total_seconds() / 3600
        # 根据duration和power生成demand列
        df1['demand'] = df1['duration'] * df1['charge_speed_station']
        return df1

    def cal_power_avg(charging_orders1):
        dates = charging_orders1['stime'].dt.date
        # 找出所有不同的日期
        unique_dates = dates.unique()
        charging_orders2=charging_orders1[charging_orders1["stime"].dt.date==unique_dates[1]]
        charging_orders2_agg=charging_orders2.groupby(["station_id","lon","lat"])["demand"].sum().reset_index()
        charging_orders2_agg["power_avg"]=charging_orders2_agg["demand"]/24
        return charging_orders2_agg

    def cal_num(charging_orders2_agg,scalar_factor=10,gum_avg=100, ult_rate=0.1):
        #actual_demand
        charging_orders2_agg["power_avg_actual"]=charging_orders2_agg["power_avg"]*scalar_factor
        #charge_demand
        charging_orders2_agg["power_total"]=charging_orders2_agg["power_avg_actual"]/ult_rate
        #gun_num
        charging_orders2_agg["gun_num"]=charging_orders2_agg["power_total"]/gum_avg
        charging_orders2_agg["gun_num"]=charging_orders2_agg["gun_num"].astype(int)
        return charging_orders2_agg
    
    charging_orders, waiting_orders=process_station_data(station_info_path, step_length=300)
    charging_orders1=split_day(charging_orders)
    charging_orders2_agg=cal_power_avg(charging_orders1)
    charging_orders2_agg=cal_num(charging_orders2_agg)
    return charging_orders2_agg

In [3]:
station_info_path = r"D:\北大深\华为充电桩网络优化\data\data\backend\Models\station_selection\station_infos.csv"
charging_orders2_agg=capacity(station_info_path)